### Setup

In [1]:
# import packages
import arcpy
import os
import pyodbc
import pandas as pd
import numpy as np
from sqlalchemy.engine import URL, create_engine
from arcgis import GIS

# setup sql connection
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde;UID=sde;PWD=staff"
connection_url    = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
conn              = create_engine(connection_url)

# set workspace
arcpy.env.workspace = "F:\GIS\PROJECTS\Transportation\Equity\Equity.gdb"
workspace           = "F:\GIS\PROJECTS\Transportation\Equity\Equity.gdb"
workspace_folder    = r"F:\GIS\PROJECTS\Transportation\Equity"
# set overwrite to true
arcpy.env.overwriteOutput = True

# in memory output file path
memory_workspace = "memory" + "\\"

# set local variables
# census data
censusFeatures = "F:\GIS\PROJECTS\Transportation\Equity\Vector.sde\sde.SDE.Census\sde.SDE.Tahoe_Census_Geography"
censusTable    = "F:\GIS\PROJECTS\Transportation\Equity\Vector.sde\sde.SDE.Census_Demographics"

# local gdb census data pre-processed
dec2010  = "Tahoe_Census_2010"
dec2020  = "Tahoe_Census_2020"
acs2020  = "Tahoe_ACS_2020"
zvh      = 'ZVH_2020_ACS'
seniors  = 'Age_2020ACS'
disabled = 'HH_Disability_2020ACS'
poverty  = 'Poverty_2020ACS'
indisable= 'Individual_Disability_2020ACS'

# locations data
grocery  = "FreshFoodLocations"
medicare = "HealthCareLocations"
freebeach= 'Free_Beaches'
beaches  = "Public_Beaches"
meetings = "PublicMeetingLocations"

# slope classes
slope    = "C:\GIS\DB_CONNECT\Raster.sde\sde_imagery.SDE.Slope_Reclassified"
zonalstat_slopeClass = "ZonalStat_Hex_SlopeClassTable"

# network dataset info
nd_path =  r"F:\GIS\GIS_DATA\Transportation\Basemap Features\Roads\Streets Network Dataset\Streets_NEW_ND.gdb\Streets_SDC_ND\Streets_CA_NV_ND"
nd_layer_name = "Streets_CA_NV"

# transportation network data
streets    = "F:\GIS\PROJECTS\Transportation\Equity\Vector.sde\sde.SDE.Transportation\sde.SDE.Streets"
urban      = "F:\GIS\PROJECTS\Transportation\Equity\Vector.sde\sde.SDE.Jurisdictions\sde.SDE.UrbanAreas"

# service areas
freebeachSA= os.path.join(workspace,"Free_Beach_ServiceArea")
beachSA    = os.path.join(workspace,"Public_Beach_ServiceArea")
grocerySA  = os.path.join(workspace,"Fresh_Food_ServiceArea")
medicareSA = os.path.join(workspace,"Medicare_ServiceArea")
meetingSA  = os.path.join(workspace,"Public_Meetings_ServiceArea")

# crash data
tahoeCrash   = r"F:\GIS\PROJECTS\Transportation\Equity\Vector.sde\sde.SDE.Transportation\sde.SDE.Tahoe_Crash"
crashFatal   = arcpy.MakeFeatureLayer_management(tahoeCrash, "Fatal_Crash_Layer", "Crash_Severity = 'Fatal'")
crashSevere  = arcpy.MakeFeatureLayer_management(tahoeCrash, "Severe_Crash_Layer", "Crash_Severity = 'Severe injury'")
crashOther   = arcpy.MakeFeatureLayer_management(tahoeCrash, "Other_Crash_Layer", "Crash_Severity IN ('Complaint of pain', 'Other visible injury', 'Property damage only')")
crashBikePed = arcpy.MakeFeatureLayer_management(tahoeCrash, "BikePed_Crash_Layer", "Bicycle_Involved = 'Y' And Pedestrian_Involved = 'Y'")

# hex bins tp aggregate data to
tahoeHex1km   = "TahoeTessellation_1sqkm"
tahoeHex01km  = "TahoeTessellation_0_1sqkm"

# hex bins to store in memory joins
hexCensus   = memory_workspace + "Hex_Census"
hexZVH      = memory_workspace + "Hex_ZVH"
hexSenior   = memory_workspace + "Hex_SENIOR"
hexDisabled = memory_workspace + "Hex_DISABLED"
hexPoverty  = memory_workspace + "Hex_POVERTY"
# 
hexBeachSA    = memory_workspace + "Hex_BeachSA"
hexGrocerySA  = memory_workspace + "Hex_GrocerySA"
hexMedicareSA = memory_workspace + "Hex_MedicareSA"
hexMeetingSA  = memory_workspace + "Hex_MeetingSA"

# hex for crash data aggregations
hexFatal    = memory_workspace + "Hex_Fatal_Crash"
hexSevere   = memory_workspace + "Hex_Severe_Crash"
hexMinor    = memory_workspace + "Hex_Minor_Crash"
hexBikePed  = memory_workspace + "Hex_BikePed_Crash"

In [10]:
# functions
# transfer attributes from spatial join feature class to tessellation
def fieldJoinCalc(updateFC, updateFieldsList, sourceFC, sourceFieldsList):
    from time import strftime  
    print ("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
#     log.info("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
    # Use list comprehension to build a dictionary from arcpy SearchCursor  
    valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(sourceFC, sourceFieldsList)}  
    with arcpy.da.UpdateCursor(updateFC, updateFieldsList) as updateRows:  
        for updateRow in updateRows:  
            # store the Join value of the row being updated in a keyValue variable  
            keyValue = updateRow[0]  
            # verify that the keyValue is in the Dictionary  
            if keyValue in valueDict:  
                # transfer the value stored under the keyValue from the dictionary to the updated field.  
                updateRow[1] = valueDict[keyValue][0]  
                updateRows.updateRow(updateRow)    
    del valueDict  
    print ("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))

# create service area
def createServiceArea(nd_path, nd_layer_name):
    # Check out the Network Analyst extension license.
    arcpy.CheckOutExtension("network")

    # Create a network dataset layer. The layer will be referenced using its name.
    arcpy.nax.MakeNetworkDatasetLayer(nd_path, nd_layer_name)

    # Instantiate a ServiceArea analysis object.
    service_area = arcpy.nax.ServiceArea(nd_layer_name)

    # Get the desired travel mode for the analysis.
    nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
    travel_mode = nd_travel_modes["New Travel Mode"]

    # Set properties.
    service_area.distanceUnits     = arcpy.nax.DistanceUnits.Miles
    service_area.defaultImpedanceCutoffs = [402.336,804.672,1609.344,3218.688, 6400]
    service_area.travelMode        = travel_mode
    service_area.allowAutoRelocate = True
    service_area.outputType        = arcpy.nax.ServiceAreaOutputType.Polygons
    service_area.geometryAtOverlap = arcpy.nax.ServiceAreaOverlapGeometry.Split
    service_area.searchTolerance   = 5000
    return service_area

# solve service area
def solveServiceArea(input_facilities, output_polygons, service_area):
    arcpy.management.Delete(output_polygons)
#     createServiceArea(nd_path,nd_layer_name)
    # load facilities
    service_area.load(arcpy.nax.ServiceAreaInputDataType.Facilities, 
                  input_facilities)
    
    # Solve the analysis.
    result = service_area.solve()
    # Export the results to a feature class. If the analysis failed print all the 
    # messages.
    if result.solveSucceeded:
        result.export(arcpy.nax.ServiceAreaOutputDataType.Polygons, output_polygons)
        print (f"Service Area generated for {input_facilities} and saved as {output_polygons}")
    else:
        arcpy.AddError("Analysis failed")
        # Print all the warning messages.
        for message in result.solverMessages(arcpy.nax.MessageSeverity.Warning):
            arcpy.AddWarning(message[-1])
        # Print all the error messages.
        for message in result.solverMessages(arcpy.nax.MessageSeverity.Error):
            arcpy.AddError(message[-1])


## Analysis

### Census Data Processing

In [60]:
# get Block Group Values
# read in census table
dfCensus      = pd.read_sql('SELECT * FROM sde.SDE.Census_Demographics', conn)

# filter for the 2020 Block Groups
dfCensus.loc[(dfCensus.sample_level == 'block group') & (dfCensus.year_sample == 2021)]

# flatten census table to values
pivotCensus = pd.pivot_table(dfCensus,index=['TRPAID'],
                              columns='variable_name',
                              values=['value'], 
                              aggfunc=np.sum,fill_value=0)

# flatten out pivoted values
flattened = pd.DataFrame(pivotCensus.to_records())

# rename columns
df = flattened.rename(columns={
    "('value', 'Total Renter Occupied Households No Vehicle Available')":"Renter_Zero_Vehicle_Households",
    "('value', 'Total Owner Occupied Households No Vehicle Available')":"Owner_Zero_Vehicle_Households",
    "('value', 'Below Poverty Status (Individuals)')":'Individual_Poverty',
    "('value', 'Poverty Status of Households Below Poverty level')":'Household_Poverty',
    "('value', 'Total Disability Population')":'Total_Disabled',
    "('value', '65 to 74')": "Over_65",
    "('value', 'Over 75')" : "Over_75",
    "('value', 'Total Households')":'Total_Households',
    "('value', 'Total')":'Total_Population'
                                })

df["Zero_Vehicle_Households"]      = df["Owner_Zero_Vehicle_Households"]+df["Renter_Zero_Vehicle_Households"]
df['Total_Seniors']                = df["Over_65"]+df['Over_75']
df['Percent_Individual_Poverty']   = (df['Individual_Poverty']/df['Total_Population'])*100
df['Percent_Household_Poverty']    = (df['Household_Poverty']/df['Total_Households'])*100
df['Percent_Senior']               = (df['Total_Seniors']/df['Total_Population'])*100
df['Percent_Disabled']             = (df['Total_Disabled']/df['Total_Population'])*100
df['Percent_ZeroVehicleHousehold'] =(df['Zero_Vehicle_Households']/df['Total_Households'])*100
# pair down fields
dfNew = df[["TRPAID",
            'Individual_Poverty','Household_Poverty',
            "Total_Disabled","Zero_Vehicle_Households",
            "Total_Seniors","Percent_Senior", "Percent_Disabled",
            "Percent_Individual_Poverty","Percent_Household_Poverty",
            "Percent_ZeroVehicleHousehold",            
            "Total_Households", "Total_Population"]]

# set index
dfNew.set_index('TRPAID')

# export table to workspace geodatabase
dfNew.spatial.to_table(os.path.join(workspace,"CensusValues_BlockGroup_ACS2021"))
# # export to table
# dfNew.to_csv(os.path.join(workspace_folder, "CensusValues_BlockGroup_ACS2021.csv"))

'F:\\GIS\\PROJECTS\\Transportation\\Equity\\Equity.gdb\\CensusValues_BlockGroup_ACS2021'

In [61]:
# join new table to feature class
censusValues = os.path.join(workspace,"CensusValues_BlockGroup_ACS2021")
censusBlockGroups = arcpy.MakeFeatureLayer_management(censusFeatures, "Census_BlockGroup_Layer", 
                                                      "Geography = 'Block Group' AND Year = 2020")
# join to features
censusBlockGroupValues    = arcpy.management.AddJoin(censusBlockGroups, "TRPAID",censusValues, "TRPAID")

# copy out new feature class
arcpy.management.CopyFeatures(censusBlockGroupValues, "TahoeCensus_BlockGroups_2020")

<Result 'F:\\GIS\\PROJECTS\\Transportation\\Equity\\Equity.gdb\\TahoeCensus_BlockGroups_2020'>

### Generate Tessellations

In [42]:
arcpy.management.GenerateTessellation(
    Output_Feature_Class="TahoeTessellation_0_1sqkm",
    Extent='-13388021.2623077 4678916.64488463 -13342410.4544964 4769413.84969433 PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]',
    Shape_Type="HEXAGON",
    Size="0.1 SquareKilometers",
    Spatial_Reference='PROJCS["NAD_1983_UTM_Zone_10N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-123.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5120900 -9998100 10000.0016462827;-100000 10000;-100000 10000;1.00000016391277E-03;0.001;0.001;IsHighPrecision',
    H3_Resolution=7
)

arcpy.management.SelectLayerByLocation(
    in_layer="TahoeTessellation_0_1sqkm",
    overlap_type="INTERSECT",
    select_features="TRPABoundary_Erase",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="INVERT"
)

arcpy.management.DeleteRows(
    in_rows="TahoeTessellation_0_1sqkm"
)

<Result 'TahoeTessellation_0_1sqkm'>

In [43]:
arcpy.management.GenerateTessellation(
    Output_Feature_Class="TahoeTessellation_1sqkm",
    Extent="737668.911313948 4288193.03147232 770952.921734459 4357302.88519489",
    Shape_Type="TRANSVERSE_HEXAGON",
    Size="1 SquareKilometers",
    Spatial_Reference='PROJCS["NAD_1983_UTM_Zone_10N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-123.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5120900 -9998100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision'
)

arcpy.management.SelectLayerByLocation(
    in_layer="TahoeTessellation_1sqkm",
    overlap_type="INTERSECT",
    select_features="TRPABoundary_Erase",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="INVERT"
)

arcpy.management.DeleteRows(
    in_rows= "TahoeTessellation_1sqkm"
)


<Result 'TahoeTessellation_1sqkm'>

### Add Fields

In [44]:
# Add fields to accept join values
 
# Set local variables
# inFeatures = "TahoeTessellation_1sqkm"
inFeatures = tahoeHex01km
fieldPrecision = 9

fieldName1  = "DISABILITY_DENSITY"
fieldAlias1 = "Disability Density"

fieldName2  = "POVERTY_DENSITY"
fieldAlias2 = "Poverty Density"

fieldName3  = "ZEROVHH_DENSITY"
fieldAlias3 = "Zero-Vehicle Household Density"

fieldName4  = "SENIOR_DENSITY"
fieldAlias4 = "Senior Density"

fieldName5  = "FOOD_ACCESS_DISTANCE"
fieldAlias5  = "Distance to Food Access"

fieldName6  = "BEACH_ACCESS_DISTANCE"
fieldAlias6 = "Distance to Free Public Beach"

fieldName7  = "MEDICARE_ACCESS_DISTANCE"
fieldAlias7 = "Distance to Medicare Facility"

fieldName8  = "MEETING_ACCESS_DISTANCE"
fieldAlias8 = "Distance to Public Meeting Location"

fieldName9  = "TRANSIT_ACCESS_DISTANCE"
fieldAlias9 = "Distance to Transit Stop"

fieldName10 = "SEVERE_CRASH_COUNT"
fieldAlias10= "Number of Severe Crashes"

fieldName11 = "FATAL_CRASH_COUNT"
fieldAlias11= "Number of Fatal Crashes"

fieldName12 = "OTHER_CRASH_COUNT"
fieldAlias12= "Number of Minor Crashes"

fieldName21 = "BIKEPED_CRASH_COUNT"
fieldAlias21= "Number of Bike/Ped Crashes"

fieldName13  = "DISABILITY_TOTAL"
fieldAlias13 = "Disability Total"

fieldName14  = "POVERTY_TOTAL"
fieldAlias14 = "Poverty Total"

fieldName15  = "ZEROVHH_TOTAL"
fieldAlias15 = "Zero-Vehicle Household Total"

fieldName16  = "SENIOR_TOTAL"
fieldAlias16 = "Senior Total"

fieldName17  = "DISABILITY_PERCENT"
fieldAlias17 = "Disability Percent"

fieldName18  = "POVERTY_PERCENT"
fieldAlias18 = "Poverty Percent"

fieldName19  = "ZEROVHH_PERCENT"
fieldAlias19 = "Zero-Vehicle Household Percent"

fieldName20  = "SENIOR_PERCENT"
fieldAlias20 = "Senior Percent"

### Density Fields
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName1, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias1, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName2, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias2, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName3, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias3, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName4, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias4, 
                          field_is_nullable="NULLABLE")
### Distance From Fields
# Run AddField
arcpy.management.AddField(inFeatures, 
                          fieldName5, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias5, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName6, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias6, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName7, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias7, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName8, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias8, 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName9, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias9, 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName10, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias10, 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName11, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias11, 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName12, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias12, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName13, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias13, 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName14, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias14, 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName15, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias15, 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName16, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias16, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName17, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias17, 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName18, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias18, 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName19, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias19, 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName20, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias20, 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName21, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias21, 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          "MAJORITY_SLOPE_CLASS", "SHORT", 
                          field_alias="Majority Slope Class")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          "IS_URBAN", "TEXT", 
                          field_alias="Within Urban Area Boundary?", 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          "HAS_ROAD", "TEXT", 
                          field_alias="Road within Hex?", 
                          field_is_nullable="NULLABLE")


<Result 'TahoeTessellation_0_1sqkm'>

### Spatial Joins & Zonal Stats

In [11]:
# zonal stats as table - hex bin & slope class
arcpy.ia.ZonalStatisticsAsTable(
    in_zone_data=tahoeHex01km ,
    zone_field="GRID_ID",
    in_value_raster=slope,
    out_table=zonalstat_slopeClass,
    ignore_nodata="DATA",
    statistics_type="MAJORITY"
)

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','MAJORITY_SLOPE_CLASS'], 
              zonalstat_slopeClass,  ['GRID_ID','MAJORITY'])

### Census Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, "TahoeCensus_BlockGroups_2020", hexCensus, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','DISABILITY_TOTAL'], 
              hexCensus,    ['GRID_ID','CensusValues_BlockGroup_ACS2021_Total_Disabled'])
# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','POVERTY_TOTAL'], 
              hexCensus,    ['GRID_ID','CensusValues_BlockGroup_ACS2021_Individual_Poverty'])
# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','ZEROVHH_TOTAL'], 
              hexCensus,    ['GRID_ID','CensusValues_BlockGroup_ACS2021_Zero_Vehicle_Households'])
# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','SENIOR_TOTAL'], 
              hexCensus,    ['GRID_ID','CensusValues_BlockGroup_ACS2021_Total_Seniors'])

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','SENIOR_PERCENT'], 
              hexCensus,    ['GRID_ID','CensusValues_BlockGroup_ACS2021_Percent_Senior'])
# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','DISABILITY_PERCENT'], 
              hexCensus,    ['GRID_ID','CensusValues_BlockGroup_ACS2021_Percent_Disabled'])
# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','POVERTY_PERCENT'], 
              hexCensus,    ['GRID_ID','CensusValues_BlockGroup_ACS2021_Percent_Individual_Poverty'])
# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','ZEROVHH_PERCENT'], 
              hexCensus,    ['GRID_ID','CensusValues_BlockGroup_ACS2021_Percent_ZeroVehicleHousehold'])

### Zero Vehicle Household Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, zvh, hexZVH, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','ZEROVHH_DENSITY'], 
              hexZVH,  ['GRID_ID','ZVH_DENS'])

### Seniors Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, seniors, hexSenior, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','SENIOR_DENSITY'], 
              hexSenior,  ['GRID_ID','OVER65_DENS'])

### Disabled Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, disabled, hexDisabled, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','DISABILITY_DENSITY'], 
              hexDisabled,  ['GRID_ID','HH_DISABILITY_DENS'])

### Poverty Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, poverty, hexPoverty, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','POVERTY_DENSITY'], 
              hexPoverty,  ['GRID_ID','HH_POVERTY_DENS'])

### Crash Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, crashFatal, 
                           hexFatal, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "CONTAINS", "", "")

arcpy.SpatialJoin_analysis(tahoeHex01km, crashSevere, 
                           hexSevere, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "CONTAINS", "", "")

arcpy.SpatialJoin_analysis(tahoeHex01km, crashOther, 
                           hexMinor, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "CONTAINS", "", "")

arcpy.SpatialJoin_analysis(tahoeHex01km, "BikePed_Crash_Layer", 
                           hexBikePed, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "CONTAINS", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','FATAL_CRASH_COUNT'], 
              hexFatal,     ['GRID_ID','Join_Count'])
# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','SEVERE_CRASH_COUNT'], 
              hexSevere,    ['GRID_ID','Join_Count'])
# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','OTHER_CRASH_COUNT'], 
              hexMinor,     ['GRID_ID','Join_Count'])
# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','BIKEPED_CRASH_COUNT'], 
              hexBikePed,   ['GRID_ID','Join_Count'])

### Transit Stop Access Update --------------------------------------------------------------------------------------###
hexTransit = memory_workspace + "Hex_Transit_Walksheds"
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, "Transit_Walksheds", hexTransit, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

fieldJoinCalc(tahoeHex01km, ["GRID_ID", "TRANSIT_ACCESS_DISTANCE"],
              hexTransit, ["GRID_ID", "Distance"])
### Beach Access Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, freebeachSA, hexBeachSA, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','BEACH_ACCESS_DISTANCE'], 
              hexBeachSA,  ['GRID_ID','ToBreak'])

### Grocery Access Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, grocerySA, hexGrocerySA, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','FOOD_ACCESS_DISTANCE'], 
              hexGrocerySA,  ['GRID_ID','ToBreak'])

### Medicare Access Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, medicareSA, hexMedicareSA, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','MEDICARE_ACCESS_DISTANCE'], 
              hexMedicareSA,  ['GRID_ID','ToBreak'])

### Public Meeting Access Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, meetingSA, hexMeetingSA, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','MEETING_ACCESS_DISTANCE'], 
              hexMeetingSA,  ['GRID_ID','ToBreak'])

### Urban Hex Attribute Update -----------------------------------------------------------------------------###

# Select parcels that intersect urban area boundary
urbanSelect = arcpy.SelectLayerByLocation_management(tahoeHex01km, 
                                                          'INTERSECT', 
                                                           urban, 
                                                           0, 
                                                          'NEW_SELECTION')
# Update field 1= yes 0 = no
with arcpy.da.UpdateCursor(urbanSelect, ['IS_URBAN']) as cursor:
    for row in cursor:
        row[0] = '1'
        cursor.updateRow(row) 
del cursor 

# switch the selection
urbanAreaSelect = arcpy.SelectLayerByAttribute_management(urbanSelect,'SWITCH_SELECTION')

# update other parcels
with arcpy.da.UpdateCursor(urbanAreaSelect, ['IS_URBAN']) as cursor:
    for row in cursor:
        row[0] = '0'
        cursor.updateRow(row)
del cursor

### Streets Hex Attribute Update -----------------------------------------------------------------------------###

# Select parcels that intersect urban area boundary
streetSelect = arcpy.SelectLayerByLocation_management(tahoeHex01km, 
                                                          'INTERSECT', 
                                                           streets, 
                                                           0, 
                                                          'NEW_SELECTION')
# Update field 1= yes 0 = no
with arcpy.da.UpdateCursor(streetSelect, ['HAS_ROAD']) as cursor:
    for row in cursor:
        row[0] = '1'
        cursor.updateRow(row) 
del cursor 

# switch the selection
streetNotSelect = arcpy.SelectLayerByAttribute_management(streetSelect,'SWITCH_SELECTION')

# update other parcels
with arcpy.da.UpdateCursor(streetNotSelect, ['HAS_ROAD']) as cursor:
    for row in cursor:
        row[0] = '0'
        cursor.updateRow(row)
del cursor

# clear selection
arcpy.SelectLayerByAttribute_management(tahoeHex01km, "CLEAR_SELECTION")

Started data transfer: 2023-07-26 15:05:08
Finished data transfer: 2023-07-26 15:05:10
Started data transfer: 2023-07-26 15:05:30
Finished data transfer: 2023-07-26 15:05:31
Started data transfer: 2023-07-26 15:05:31
Finished data transfer: 2023-07-26 15:05:33
Started data transfer: 2023-07-26 15:05:33
Finished data transfer: 2023-07-26 15:05:34
Started data transfer: 2023-07-26 15:05:34
Finished data transfer: 2023-07-26 15:05:35
Started data transfer: 2023-07-26 15:05:35
Finished data transfer: 2023-07-26 15:05:36
Started data transfer: 2023-07-26 15:05:36
Finished data transfer: 2023-07-26 15:05:37
Started data transfer: 2023-07-26 15:05:37
Finished data transfer: 2023-07-26 15:05:38
Started data transfer: 2023-07-26 15:05:38
Finished data transfer: 2023-07-26 15:05:39
Started data transfer: 2023-07-26 15:05:56
Finished data transfer: 2023-07-26 15:05:58
Started data transfer: 2023-07-26 15:06:16
Finished data transfer: 2023-07-26 15:06:19
Started data transfer: 2023-07-26 15:06:40


<Result 'TahoeTessellation_0_1sqkm_La6'>

### Transform Values

In [23]:
# If the value is null, replace it with 0
with arcpy.da.UpdateCursor(tahoeHex01km, ['ZEROVHH_DENSITY', 'SENIOR_DENSITY', 
                                          'POVERTY_DENSITY', 'DISABILITY_DENSITY']) as cursor:
    for row in cursor:
        if row[0] is None:
            row[0] = 0
            cursor.updateRow(row)
        if row[1] is None:
            row[1] = 0
            cursor.updateRow(row)
        if row[2] is None:
            row[2] = 0
            cursor.updateRow(row)
        if row[3] is None:
            row[3] = 0
            cursor.updateRow(row)
del cursor

In [24]:
with arcpy.da.UpdateCursor(tahoeHex01km, ["BEACH_ACCESS_DISTANCE"]) as cursor:
    for row in cursor:
        if row[0]  == 402.336:
            row[0]  = 0.25
        elif row[0]  == 804.672:
            row[0]  = 0.5
        elif row[0]  == 1609.344:
            row[0]  = 1
        elif row[0] is None:
            row[0] = 3
        elif row[0]  > 1609.344 and row[0] <= 3218.688:
            row[0]  = 2
        elif row[0] > 3218.688 and row[0] <= 6400:
            row[0]= 3
        else:
            row[0] = 3

        cursor.updateRow(row)
del cursor

with arcpy.da.UpdateCursor(tahoeHex01km, ["FOOD_ACCESS_DISTANCE"]) as cursor:
    for row in cursor:
        if row[0]  == 402.336:
            row[0]  = 0.25
        elif row[0]  == 804.672:
            row[0]  = 0.5
        elif row[0]  == 1609.344:
            row[0]  = 1
        elif row[0] is None:
            row[0] = 3
        elif row[0]  > 1609.344 and row[0] <= 3218.688:
            row[0]  = 2
        elif row[0] > 3218.688 and row[0] <= 6400:
            row[0]= 3
        else:
            row[0] = 3

        cursor.updateRow(row)
del cursor

with arcpy.da.UpdateCursor(tahoeHex01km, ["MEDICARE_ACCESS_DISTANCE"]) as cursor:
    for row in cursor:
        if row[0]  == 402.336:
            row[0]  = 0.25
        elif row[0]  == 804.672:
            row[0]  = 0.5
        elif row[0]  == 1609.344:
            row[0]  = 1
        elif row[0] is None:
            row[0] = 3
        elif row[0]  > 1609.344 and row[0] <= 3218.688:
            row[0]  = 2
        elif row[0] > 3218.688 and row[0] <= 6400:
            row[0]= 3
        else:
            row[0] = 3

        cursor.updateRow(row)
del cursor

with arcpy.da.UpdateCursor(tahoeHex01km, ["MEETING_ACCESS_DISTANCE"]) as cursor:
    for row in cursor:
        if row[0]  == 402.336:
            row[0]  = 0.25
        elif row[0]  == 804.672:
            row[0]  = 0.5
        elif row[0]  == 1609.344:
            row[0]  = 1
        elif row[0] is None:
            row[0] = 3
        elif row[0]  > 1609.344 and row[0] <= 3218.688:
            row[0]  = 2
        elif row[0] > 3218.688 and row[0] <= 6400:
            row[0]= 3
        else:
            row[0] = 3

        cursor.updateRow(row)
del cursor

# with arcpy.da.UpdateCursor(tahoeHex01km, ["MAJORITY_SLOPE_CLASS"]) as cursor:
#     for row in cursor:
#         if row[0] == "1":
#             row[0]  = "0-5%"
#         elif row[0]  == "2":
#             row[0]  = "5-15%"
#         elif row[0]  == "3":
#             row[0]  = "15-30%"
#         elif row[0]  == "4":
#             row[0]  = "30-50%"
#         elif row[0]  == "5":
#             row[0]  = ">50%"
#         cursor.updateRow(row)
# del cursor

### Network Analysis

#### Create Service Area

In [9]:
createServiceArea(nd_path, nd_layer_name)

#### Solve Service Area

In [20]:
# Check out the Network Analyst extension license.
arcpy.CheckOutExtension("network")

# Create a network dataset layer. The layer will be referenced using its name.
arcpy.nax.MakeNetworkDatasetLayer(nd_path, nd_layer_name)

# Instantiate a ServiceArea analysis object.
service_area = arcpy.nax.ServiceArea(nd_layer_name)

# Get the desired travel mode for the analysis.
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["New Travel Mode"]

# Set properties.
service_area.distanceUnits     = arcpy.nax.DistanceUnits.Miles
service_area.defaultImpedanceCutoffs = [402.336,804.672,1609.344,3218.688, 6400]
service_area.travelMode        = travel_mode
service_area.allowAutoRelocate = True
service_area.outputType        = arcpy.nax.ServiceAreaOutputType.Polygons
service_area.geometryAtOverlap = arcpy.nax.ServiceAreaOverlapGeometry.Split
service_area.searchTolerance   = 5000

In [21]:

# solveServiceArea(freebeach,freebeachSA, service_area)
# solveServiceArea(grocery,grocerySA, service_area)
solveServiceArea(meetings,meetingSA, service_area)
# solveServiceArea(medicare,medicareSA, service_area)

Service Area generated for PublicMeetingLocations and saved as F:\GIS\PROJECTS\Transportation\Equity\Equity.gdb\Public_Meetings_ServiceArea


### Safety Analysis

In [19]:
tahoeHexRoad = arcpy.MakeFeatureLayer_management(tahoeHex01km, "Hex_Road_Layer", 
                        "HAS_ROAD = '1'")
arcpy.stats.HotSpots(
    Input_Feature_Class=tahoeHexRoad,
    Input_Field="FATAL_CRASH_COUNT",
    Output_Feature_Class=r"TahoeTessellation_FatalCrash_HotSpots",
    Conceptualization_of_Spatial_Relationships="FIXED_DISTANCE_BAND",
    Distance_Method="EUCLIDEAN_DISTANCE",
    Standardization="ROW",
    Distance_Band_or_Threshold_Distance=None,
    Self_Potential_Field=None,
    Weights_Matrix_File=None,
    Apply_False_Discovery_Rate__FDR__Correction="NO_FDR",
    number_of_neighbors=None
)

<Result 'F:\\GIS\\PROJECTS\\Transportation\\Equity\\Equity.gdb\\TahoeTessellation_FatalCrash_HotSpots'>

In [18]:
tahoeHexRoad = arcpy.MakeFeatureLayer_management(tahoeHex01km, "Hex_Road_Layer", 
                        "HAS_ROAD = '1'")
arcpy.stats.HotSpots(
    Input_Feature_Class=tahoeHexRoad,
    Input_Field="SEVERE_CRASH_COUNT",
    Output_Feature_Class=r"TahoeTessellation_FatalCrash_HotSpots",
    Conceptualization_of_Spatial_Relationships="FIXED_DISTANCE_BAND",
    Distance_Method="EUCLIDEAN_DISTANCE",
    Standardization="ROW",
    Distance_Band_or_Threshold_Distance=None,
    Self_Potential_Field=None,
    Weights_Matrix_File=None,
    Apply_False_Discovery_Rate__FDR__Correction="NO_FDR",
    number_of_neighbors=None
)

<Result 'F:\\GIS\\PROJECTS\\Transportation\\Equity\\Equity.gdb\\TahoeTessellation_FatalCrash_HotSpots'>

### Equity Index

#### Composite Index

In [16]:
tahoeHexUrban = arcpy.MakeFeatureLayer_management(tahoeHex01km, "Hex_Urban_Layer", 
                        "IS_URBAN = '1' Or HAS_ROAD = '1'")
 

with arcpy.EnvManager(scratchWorkspace=r"F:\GIS\PROJECTS\Transportation\Equity\Equity.gdb", workspace=r"F:\GIS\PROJECTS\Transportation\Equity\Equity.gdb"):
    arcpy.stats.CalculateCompositeIndex(
        in_table=tahoeHexUrban,
        in_variables="DISABILITY_DENSITY #;MEDICARE_ACCESS_DISTANCE false;FOOD_ACCESS_DISTANCE false;MAJORITY_SLOPE_CLASS #",
        append_to_input="NEW_FEATURES",
        out_table="TahoeTessellation_0_1sqkm_CalculateCompositeIndex",
        index_preset="MEAN_SCALED",
        preprocessing="MINMAX",
        pre_threshold_scaling="THRESHOLD_PERCENTILE",
        pre_custom_zscore=None,
        pre_min_max=None,
        pre_thresholds=None,
        index_method="MEAN",
        index_weights="DISABILITY_DENSITY 1;MEDICARE_ACCESS_DISTANCE 1;FOOD_ACCESS_DISTANCE 1;MAJORITY_SLOPE_CLASS 1",
        out_index_name="",
        out_index_reverse=None,
        post_min_max=None,
        post_reclass=None,
        post_num_classes=5,
        post_custom_classes=None
    )